In [1]:
from __future__ import absolute_import, division, print_function

import numpy as np
import xlrd 
from bert_serving.client import BertClient

import os
import sst
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from torch_rnn_classifier import TorchRNNClassifier
from sklearn.metrics import classification_report

In [2]:
bc = BertClient(check_length=False)

In [12]:
print("Training data")
book = xlrd.open_workbook("gender_training.xlsx")
sheet = book.sheet_by_index(0)

X_str_train = []
y_train = []

for row in range(2,24756):
    first = sheet.cell_value(row,1)
    first = first.lower()
    last = sheet.cell_value(row,2)
    last = last.lower()
    name = first + " " + last
    gender = sheet.cell_value(row,11)
    gender = gender.lower()
    X_str_train.append(name)
    y_train.append(gender)

print(X_str_train)
print(y_train)

Training data
['liz abdul', 'marc aafjes', 'joshua aaron', 'sanford aaron', 'zachary aarons', 'liat aaronson', 'benjamin abadi', 'mike abadi', 'essam abadir', 'sam abadir', 'majid abai', 'omar abbara', 'sam abbitt', 'joel abbo', 'cooper abbott', 'curtis abbott', 'jeffrey abbott', 'james abbott', 'kelly abbott', 'michael abbott', 'steve abbott', 'todd abbott', 'bryan abboud', 'mahmoud abdallah', 'menna abdelrahman', 'gary abrahams', 'behrooz abdi', 'mark abdou', 'aamer abdullah', 'firdavs abdunazarov', 'anthony abdy', 'arie abecassis', 'gabriel abed', 'christopher abel', 'godard abel', 'james abeler', 'john abeles', 'jean-pierre abello', 'kenneth abeloe', 'jonathan aberman', 'richard aberman', 'john abernathy', 'john abes', 'jacky abitbol', 'sasha ablovatskiy', 'mona aboelnaga', 'alexander abolmasov', 'hazem abolrous', 'jeffrey aborn', 'fadi abouabsi', 'hash aboulhosn', 'nick abouzeid', 'akash abraham', 'anthony abraham', 'david abraham', 'george abraham', 'gregory abraham', 'jack abrah

In [4]:
print(len(X_str_train))
print(len(y_train))

24754
24754


In [9]:
book1 = xlrd.open_workbook("gender_testing.xlsx")
testsheet = book1.sheet_by_index(0)

X_str_dev_red = []
y_dev_red = []

for row in range(0,6191):
    first = testsheet.cell_value(row,1)
    first = first.lower()
    last = testsheet.cell_value(row,2)
    last = last.lower()
    name = first + " " + last
    X_str_dev_red.append(name)
    gender = testsheet.cell_value(row,11)
    gender = gender.lower()
    y_dev_red.append(gender)
    
print(X_str_dev_red)
print(y_dev_red)
print(len(X_str_dev_red))
print(len(y_dev_red))


['willem schutte', 'jeffrey schutz', 'jordanna schutz', 'rob schutz', 'jason schutzbank', 'michael schutzler', 'john schveibinz', 'doug scott', 'david schwab', 'frank schwab', 'matthew schwab', 'michael schwab', 'andrew schwabe', 'andrew schwaeber', 'michael schwalb', 'chris schwalbach', 'jj schwan', 'morgan schwanke', 'robert schwartz', 'david schwartz', 'evan schwartz', 'hagi schwartz', 'howie schwartz', 'hy schwartz', 'jake schwartz', 'jonathan schwartz', 'josh schwartz', 'kathleen schwartz', 'mark schwartz', 'nimrod schwartz', 'scott schwartz', 'todd schwartz', 'yoav schwartz', 'evan schwartzfarb', 'phil schwartzman', 'fred schwarzer', 'zak schwarzman', 'kelly schwedland', 'georg schwegler', 'peter schweinfurth', 'richard schwerdtfeger', 'samuel schwerin', 'erica schweyer', 'jeremy schwimmer', 'zach schwindt', 'matthew schwingel', 'paul sciabica', 'rob sciama', 'mark scianna', 'jacques sciberras', 'donald scifres', 'michael scissons', 'gabriele scoditti', 'oscar scolari', 'melissa 

In [10]:
book3 = xlrd.open_workbook("gender_eval.xlsx")
evalsheet = book3.sheet_by_index(0)

X_str_dev = []
y_dev = []

for row in range(0,1099):
    first = evalsheet.cell_value(row,1)
    first = first.lower()
    last = evalsheet.cell_value(row,2)
    last = last.lower()
    name = first + " " + last
    X_str_dev.append(name)
    gender = evalsheet.cell_value(row,11)
    gender = gender.lower()
    y_dev.append(gender)
    
print(X_str_dev)
print(y_dev)
print(len(X_str_dev))
print(len(y_dev))


['mikaal abdulla', 'sammy abdullah', 'benzion aboud', 'akiba abraham', 'shripati acharya', 'farrhad acidwalla', 'bahigh acuna', 'taylor adams', 'tralance addy', 'dayo adefila', 'onikepe adegbola', 'laverne adekunle', 'lamide adeosun', 'sara adler', 'yael adler', 'maja adriaensen', 'bult adrian', 'alokik advani', 'asheesh advani', 'noubar afeyan', 'jo hannah afton', 'shubh agarwal', 'shan aggarwal', 'cy agley', 'renee julie elisabeth aguiar-lucander', 'anjou ahlborn', 'muttee ahmad', 'zimran ahmed', 'saemin ahn', 'oais ahsan', 'ratinder ahuja', 'osma ahvenlampi', 'adeyemi ajao', 'dara akbarian', 'akky akimoto', 'kiran akkineni', 'studio akko', 'dima alashram', 'ernstjan albers', 'tawn albright', 'aldaba aldarmaki', 'taylor alexander', 'yamandou alexander', 'sultan alhokair', 'arish ali', 'fehzan ali', 'murshid ali', 'abidali alibhai', 'areg alimian', 'casey allen', 'cory allison', 'robin allenson', 'mbwana alliy', 'erinc alper', 'mussaad al-razouki', 'sabah alsabah', 'monel amin', 'shah

In [13]:
X_bert_train, bert_train_toks = bc.encode(
    list(X_str_train), show_tokens=True)

In [14]:
print("dev")
X_bert_dev, bert_dev_toks = bc.encode(
    list(X_str_dev), show_tokens=True)


dev


In [32]:
X_bert_dev_red, bert_dev_toks_red = bc.encode(
    list(X_str_dev_red), show_tokens=True)

In [34]:
print(X_bert_dev)
print(X_bert_dev_red)

[[[-7.63114214e-01  3.15484434e-01 -3.81498456e-01 ... -1.29728261e-02
   -2.04604529e-02  3.52391116e-02]
  [-1.86638206e-01 -1.61683887e-01 -1.78324610e-01 ... -3.34501445e-01
   -9.43782032e-02 -2.74924040e-01]
  [-5.48861660e-02  6.19358644e-02 -3.81308496e-01 ... -2.88755983e-01
   -4.44952011e-01 -6.33590817e-01]
  ...
  [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00 ... -0.00000000e+00
   -0.00000000e+00 -0.00000000e+00]
  [-0.00000000e+00  0.00000000e+00 -0.00000000e+00 ... -0.00000000e+00
   -0.00000000e+00 -0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[-8.26076865e-01  6.77274644e-01 -4.61738527e-01 ... -2.42666125e-01
    3.49870116e-01 -1.44060235e-03]
  [ 1.45861898e-02 -6.54496610e-01 -4.94929194e-01 ...  1.10205099e-01
   -7.48005450e-01 -1.29966915e+00]
  [-1.07804835e-02 -2.27858320e-01 -1.17057252e+00 ... -2.06726611e-01
   -2.83089757e-01 -4.35140312e-01]
  ...
  [-0.00000000e+00  0.0

In [15]:
print(bert_dev_toks)

[['[CLS]', 'mika', '##al', 'abdul', '##la', '[SEP]'], ['[CLS]', 'sammy', 'abdullah', '[SEP]'], ['[CLS]', 'benz', '##ion', 'ab', '##oud', '[SEP]'], ['[CLS]', 'ak', '##iba', 'abraham', '[SEP]'], ['[CLS]', 'shri', '##pati', 'ac', '##har', '##ya', '[SEP]'], ['[CLS]', 'far', '##rh', '##ad', 'acid', '##wall', '##a', '[SEP]'], ['[CLS]', 'ba', '##hi', '##gh', 'ac', '##una', '[SEP]'], ['[CLS]', 'taylor', 'adams', '[SEP]'], ['[CLS]', 'tr', '##ala', '##nce', 'add', '##y', '[SEP]'], ['[CLS]', 'day', '##o', 'ad', '##ef', '##ila', '[SEP]'], ['[CLS]', 'on', '##ike', '##pe', 'ad', '##eg', '##bola', '[SEP]'], ['[CLS]', 'la', '##vern', '##e', 'ad', '##ek', '##un', '##le', '[SEP]'], ['[CLS]', 'lam', '##ide', 'ad', '##eo', '##sun', '[SEP]'], ['[CLS]', 'sara', 'adler', '[SEP]'], ['[CLS]', 'ya', '##el', 'adler', '[SEP]'], ['[CLS]', 'maj', '##a', 'ad', '##ria', '##ense', '##n', '[SEP]'], ['[CLS]', 'bu', '##lt', 'adrian', '[SEP]'], ['[CLS]', 'al', '##oki', '##k', 'ad', '##vani', '[SEP]'], ['[CLS]', 'ashe', '#

In [16]:
print(bert_dev_toks_red)

[['[CLS]', 'willem', 'sc', '##hu', '##tte', '[SEP]'], ['[CLS]', 'jeffrey', 'sc', '##hu', '##tz', '[SEP]'], ['[CLS]', 'jordan', '##na', 'sc', '##hu', '##tz', '[SEP]'], ['[CLS]', 'rob', 'sc', '##hu', '##tz', '[SEP]'], ['[CLS]', 'jason', 'sc', '##hu', '##tz', '##bank', '[SEP]'], ['[CLS]', 'michael', 'sc', '##hu', '##tz', '##ler', '[SEP]'], ['[CLS]', 'john', 'sc', '##h', '##ve', '##ib', '##in', '##z', '[SEP]'], ['[CLS]', 'doug', 'scott', '[SEP]'], ['[CLS]', 'david', 'sc', '##hwa', '##b', '[SEP]'], ['[CLS]', 'frank', 'sc', '##hwa', '##b', '[SEP]'], ['[CLS]', 'matthew', 'sc', '##hwa', '##b', '[SEP]'], ['[CLS]', 'michael', 'sc', '##hwa', '##b', '[SEP]'], ['[CLS]', 'andrew', 'sc', '##hwa', '##be', '[SEP]'], ['[CLS]', 'andrew', 'sc', '##hwa', '##eber', '[SEP]'], ['[CLS]', 'michael', 'sc', '##hwa', '##lb', '[SEP]'], ['[CLS]', 'chris', 'sc', '##hwa', '##lb', '##ach', '[SEP]'], ['[CLS]', 'jj', 'sc', '##hwa', '##n', '[SEP]'], ['[CLS]', 'morgan', 'sc', '##hwa', '##nk', '##e', '[SEP]'], ['[CLS]', 'ro

In [17]:
def bert_reduce_mean(X):
    return X.mean(axis=1)

In [18]:
X_bert_train_mean = bert_reduce_mean(X_bert_train)

In [19]:
X_bert_train_mean.shape[1]


768

In [20]:
mod = TorchShallowNeuralClassifier(
    max_iter=100, hidden_dim=300)

mod_red = TorchShallowNeuralClassifier(
    max_iter=100, hidden_dim=300)

In [21]:
%time _ = mod.fit(X_bert_train_mean, y_train)

Finished epoch 100 of 100; error is 0.0040187712365877815

CPU times: user 2min 7s, sys: 16.3 s, total: 2min 23s
Wall time: 1min 19s


In [35]:
X_bert_dev_mean = bert_reduce_mean(X_bert_dev)


In [36]:
X_bert_dev_mean_red = bert_reduce_mean(X_bert_dev_red)


In [37]:
bert_sent_preds = mod.predict(X_bert_dev_mean)

import openpyxl
xfile1 = openpyxl.load_workbook('gender_results.xlsx')
javasheetwrite = xfile1['Sheet1'] 

counter = 1

for gender in bert_sent_preds:
    index = "A" + str(counter)
    javasheetwrite[index] = gender
    counter += 1

xfile1.save('gender_results.xlsx')
    

In [39]:
bert_sent_preds_red = mod.predict(X_bert_dev_mean_red)

counter = 1

for gender in bert_sent_preds_red:
    index = "B" + str(counter)
    javasheetwrite[index] = gender
    counter += 1

xfile1.save('gender_results.xlsx')

In [40]:
print(classification_report(y_dev, bert_sent_preds, digits=3))

              precision    recall  f1-score   support

      female      0.282     0.385     0.326       148
        male      0.899     0.848     0.872       951

    accuracy                          0.785      1099
   macro avg      0.590     0.616     0.599      1099
weighted avg      0.816     0.785     0.799      1099



In [41]:
print(classification_report(y_dev_red, bert_sent_preds_red, digits=3))

              precision    recall  f1-score   support

      female      0.872     0.768     0.817       730
        male      0.970     0.985     0.977      5461

    accuracy                          0.959      6191
   macro avg      0.921     0.877     0.897      6191
weighted avg      0.958     0.959     0.958      6191

